Creating the datasets:
1. We need the games history between Home and Away as well as who won (potentially how much they won by)
2. We need the corresponding stats for that week
3. Betting data

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# Setup headless Chrome
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# Load the target URL
url = 'https://www.sportsbookreview.com/betting-odds/mlb-baseball/?date=2021-04-04'
driver.get(url)

# Wait for JS to render
time.sleep(5)

# Get the inner HTML of the <body> tag
body_html = driver.execute_script("return document.body.innerHTML;")

# Print or save to file
print(body_html)

driver.quit()


<div id="CookieBanner" data-version="1.1.0" data-template="generaxion/cookiebot-accessible-template" data-tone="neutral" name="CookieBanner" style="display: block;" class="is-visible-cookie-banner"><div id="CookieBannerOverlay"></div><div id="CookieBannerNotice" lang="en" dir="ltr" role="dialog" aria-labelledby="CookieBannerTitle" aria-describedby="CookieBannerDescription" ng-non-bindable=""><div class="cookiebanner__main"><div class="cookiebanner__main__inner"><div class="cookiebanner__main__content"><!--<p id="CookieBannerTitle" role="status" class="cookiebanner__main__title">Cookie Notice</p>--><p id="CookieBannerDescription" class="cookiebanner__main__description">Sportsbook Review uses cookies to help us provide, protect &amp; improve our services in accordance with our <a href="https://www.sportsbookreview.com/privacy-policy/" style="">Privacy &amp; Cookies</a> Policy, which includes details of how you can change your cookie settings</p></div><div class="cookiebanner__buttons"><u

In [ ]:
def extract_json_array(text, start_pattern='[{"league":"MLB"'):
    start_index = text.find(start_pattern)
    if start_index == -1:
        raise ValueError("Start pattern not found")

    bracket_count = 0
    end_index = None

    for i in range(start_index, len(text)):
        char = text[i]
        if char == "[":
            bracket_count += 1
        elif char == "]":
            bracket_count -= 1
            if bracket_count == 0:
                end_index = i + 1
                break

    if end_index is None:
        raise ValueError("Matching closing bracket not found")

    return text[start_index:end_index]


json_text = extract_json_array(body_html)

[{"league":"MLB","oddsTableModel":{"gameRows":[{"gameView":{"gameId":232914,"leagueName":"MLB","startDate":"2021-04-04T17:05:00+00:00","awayTeam":{"name":"Atlanta","fullName":"Atlanta Braves","shortName":"ATL","displayName":"Atlanta","nickname":"Braves","rank":-1},"awayTeamRotationNumber":"","awayStarter":{"firstName":"Ian","lastName":"Anderson","throwsShort":"R","firstInital":"I"},"awayTeamScore":1,"homeTeam":{"name":"Philadelphia","fullName":"Philadelphia Phillies","shortName":"PHI","displayNa


In [ ]:
import json

def print_json_tree(obj, indent=0):
    prefix = "  " * indent
    if isinstance(obj, dict):
        for key, value in obj.items():
            print(f"{prefix}{key}:")
            print_json_tree(value, indent + 1)
    elif isinstance(obj, list):
        print(f"{prefix}[{len(obj)} items]")
        if len(obj) > 0:
            print_json_tree(obj[0], indent + 1)  # Just show structure of first item
    else:
        print(f"{prefix}{type(obj).__name__}")

data = json.loads(json_text)

print_json_tree(data)

In [24]:
all_game_data = []

for game in data[0]['oddsTableModel']['gameRows']:
    gv = game['gameView']
    base = {
        'gameId': gv['gameId'],
        'startDate': gv['startDate'],
        'homeTeam': gv['homeTeam']['displayName'],
        'awayTeam': gv['awayTeam']['displayName'],
        'homePitcher': f"{gv['homeStarter']['firstName']} {gv['homeStarter']['lastName']}",
        'awayPitcher': f"{gv['awayStarter']['firstName']} {gv['awayStarter']['lastName']}",
        'homeScore': gv.get('homeTeamScore'),
        'awayScore': gv.get('awayTeamScore'),
    }

    # Grab oddsViews if available
    odds_views = game.get('oddsViews', [])
    if odds_views:
        for view in odds_views:
            if view is None:
                continue
            sportsbook = view.get("sportsbook", "unknown")

            opening = view.get("openingLine", {})
            current = view.get("currentLine", {})

            base.update({
                f"{sportsbook}_opening_homeOdds": opening.get("homeOdds"),
                f"{sportsbook}_opening_awayOdds": opening.get("awayOdds"),
                f"{sportsbook}_opening_drawOdds": opening.get("drawOdds"),
                f"{sportsbook}_opening_total": opening.get("total"),

                f"{sportsbook}_current_homeOdds": current.get("homeOdds"),
                f"{sportsbook}_current_awayOdds": current.get("awayOdds"),
                f"{sportsbook}_current_drawOdds": current.get("drawOdds"),
                f"{sportsbook}_current_total": current.get("total"),
            })

    all_game_data.append(base)

df = pd.DataFrame(all_game_data)
df

,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,fanduel_opening_homeOdds,fanduel_opening_awayOdds,...,bet365_current_drawOdds,bet365_current_total,draftkings_opening_homeOdds,draftkings_opening_awayOdds,draftkings_opening_drawOdds,draftkings_opening_total,draftkings_current_homeOdds,draftkings_current_awayOdds,draftkings_current_drawOdds,draftkings_current_total
0,232914,2021-04-04T17:05:00+00:00,Philadelphia,Atlanta,Zach Eflin,Ian Anderson,2,1,106,-124,...,0,None,108,-129,0,None,100,-120,0,None
1,232910,2021-04-04T17:05:00+00:00,NY Yankees,Toronto,Domingo German,Timothy James Zeuch,1,3,-176,148,...,0,None,-167,140,0,None,-180,150,0,None
2,232919,2021-04-04T17:10:00+00:00,Boston,Baltimore,Garrett Richards,Bruce Zimmermann,3,11,-168,142,...,0,None,-182,150,0,None,-182,150,0,None
3,232912,2021-04-04T17:10:00+00:00,Cincinnati,St. Louis,Jeff Hoffman,Carlos Martinez,12,1,-110,-106,...,0,None,-106,-109,0,None,102,-117,0,None
4,232918,2021-04-04T17:10:00+00:00,Detroit,Cleveland,Tarik Skubal,Aaron Civale,3,9,112,-132,...,0,None,112,-134,0,None,112,-130,0,None
5,232913,2021-04-04T18:10:00+00:00,Milwaukee,Minnesota,Adrian Houser,Michael Pineda,2,8,-102,-116,...,0,None,102,-118,0,None,-113,-105,0,None
6,232911,2021-04-04T18:10:00+00:00,Kansas City,Texas,Brady Singer,Jordan Lyles,3,7,-184,154,...,0,None,-182,155,0,None,-190,155,0,None
7,232920,2021-04-04T18:20:00+00:00,Chi. Cubs,Pittsburgh,Zach Davies,Mitch Keller,4,3,-180,152,...,0,None,-175,148,0,None,-186,160,0,None
8,232916,2021-04-04T19:10:00+00:00,Colorado,LA Dodgers,Austin Gomber,Julio Urias,2,4,190,-230,...,0,None,215,-250,0,None,185,-230,0,None
9,232915,2021-04-04T20:07:00+00:00,Oakland,Houston,Sean Manaea,Jose Urquidy,2,9,-110,-106,...,0,None,-114,-103,0,None,-110,-107,0,None


In [29]:
# Keep base game info
core_cols = ['gameId', 'startDate', 'homeTeam', 'awayTeam',
             'homePitcher', 'awayPitcher', 'homeScore', 'awayScore']

# Get all columns that include 'homeOdds' or 'awayOdds'
odds_cols = [col for col in df.columns if 'homeOdds' in col or 'awayOdds' in col]

# Subset the dataframe
filtered_df = df[core_cols + odds_cols]

# Preview
filtered_df

,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,fanduel_opening_homeOdds,fanduel_opening_awayOdds,...,caesars_current_homeOdds,caesars_current_awayOdds,bet365_opening_homeOdds,bet365_opening_awayOdds,bet365_current_homeOdds,bet365_current_awayOdds,draftkings_opening_homeOdds,draftkings_opening_awayOdds,draftkings_current_homeOdds,draftkings_current_awayOdds
0,232914,2021-04-04T17:05:00+00:00,Philadelphia,Atlanta,Zach Eflin,Ian Anderson,2,1,106,-124,...,100,-110,115,-135,100,-120,108,-129,100,-120
1,232910,2021-04-04T17:05:00+00:00,NY Yankees,Toronto,Domingo German,Timothy James Zeuch,1,3,-176,148,...,-190,170,-155,135,-185,165,-167,140,-180,150
2,232919,2021-04-04T17:10:00+00:00,Boston,Baltimore,Garrett Richards,Bruce Zimmermann,3,11,-168,142,...,-170,155,-170,150,-165,145,-182,150,-182,150
3,232912,2021-04-04T17:10:00+00:00,Cincinnati,St. Louis,Jeff Hoffman,Carlos Martinez,12,1,-110,-106,...,-102,-108,-105,-115,100,-120,-106,-109,102,-117
4,232918,2021-04-04T17:10:00+00:00,Detroit,Cleveland,Tarik Skubal,Aaron Civale,3,9,112,-132,...,120,-130,115,-135,115,-135,112,-134,112,-130
5,232913,2021-04-04T18:10:00+00:00,Milwaukee,Minnesota,Adrian Houser,Michael Pineda,2,8,-102,-116,...,-110,100,100,-120,-115,-105,102,-118,-113,-105
6,232911,2021-04-04T18:10:00+00:00,Kansas City,Texas,Brady Singer,Jordan Lyles,3,7,-184,154,...,-162,152,-170,150,-170,150,-182,155,-190,155
7,232920,2021-04-04T18:20:00+00:00,Chi. Cubs,Pittsburgh,Zach Davies,Mitch Keller,4,3,-180,152,...,-190,170,-165,145,-192,167,-175,148,-186,160
8,232916,2021-04-04T19:10:00+00:00,Colorado,LA Dodgers,Austin Gomber,Julio Urias,2,4,190,-230,...,190,-210,205,-245,180,-220,215,-250,185,-230
9,232915,2021-04-04T20:07:00+00:00,Oakland,Houston,Sean Manaea,Jose Urquidy,2,9,-110,-106,...,-116,106,-110,-110,-115,-105,-114,-103,-110,-107


GENERATING GAME AND ODDS

In [30]:
import os
import json
import re
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [38]:
# === Prepare folder ===
output_folder = "daily_odds"
os.makedirs(output_folder, exist_ok=True)

# === Date range ===
start_date = datetime(2022, 7, 21)
end_date = datetime(2022, 8, 20)

# === Extract JSON helper ===
def extract_json_array(text, start_pattern='[{"league":"MLB"'):
    start_index = text.find(start_pattern)
    if start_index == -1:
        raise ValueError("Start pattern not found")
    bracket_count = 0
    for i in range(start_index, len(text)):
        char = text[i]
        if char == "[":
            bracket_count += 1
        elif char == "]":
            bracket_count -= 1
            if bracket_count == 0:
                return text[start_index:i+1]
    raise ValueError("Matching closing bracket not found")

# === Scraper ===
def scrape_day(date_str):
    url = f'https://www.sportsbookreview.com/betting-odds/mlb-baseball/?date={date_str}'
    
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(6)  # allow full render
    body_html = driver.execute_script("return document.body.innerHTML;")
    driver.quit()
    
    try:
        json_text = extract_json_array(body_html)
        data = json.loads(json_text)
    except Exception as e:
        print(f"Failed on {date_str}: {e}")
        return
    
    all_game_data = []

    for game in data[0]['oddsTableModel']['gameRows']:
        gv = game['gameView']
        base = {
            'gameId': gv['gameId'],
            'startDate': gv['startDate'],
            'homeTeam': gv['homeTeam']['displayName'],
            'awayTeam': gv['awayTeam']['displayName'],
            'homePitcher': f"{gv['homeStarter']['firstName']} {gv['homeStarter']['lastName']}",
            'awayPitcher': f"{gv['awayStarter']['firstName']} {gv['awayStarter']['lastName']}",
            'homeScore': gv.get('homeTeamScore'),
            'awayScore': gv.get('awayTeamScore'),
        }

        odds_views = game.get('oddsViews', [])
        if odds_views:
            for view in odds_views:
                if view is None:
                    continue
                sportsbook = view.get("sportsbook", "unknown")
                opening = view.get("openingLine", {})
                current = view.get("currentLine", {})
                base.update({
                    f"{sportsbook}_opening_homeOdds": opening.get("homeOdds"),
                    f"{sportsbook}_opening_awayOdds": opening.get("awayOdds"),
                    f"{sportsbook}_current_homeOdds": current.get("homeOdds"),
                    f"{sportsbook}_current_awayOdds": current.get("awayOdds"),
                })

        all_game_data.append(base)

    if not all_game_data:
        print(f"No games found on {date_str}")
        return

    df = pd.DataFrame(all_game_data)

    core_cols = ['gameId', 'startDate', 'homeTeam', 'awayTeam',
                 'homePitcher', 'awayPitcher', 'homeScore', 'awayScore']
    odds_cols = [col for col in df.columns if 'homeOdds' in col or 'awayOdds' in col]
    filtered_df = df[core_cols + odds_cols]

    output_path = os.path.join(output_folder, f"{date_str}.csv")
    filtered_df.to_csv(output_path, index=False)
    print(f"Saved {date_str}")

# === Run loop ===
current = start_date
while current <= end_date:
    date_str = current.strftime("%Y-%m-%d")
    output_path = os.path.join(output_folder, f"{date_str}.csv")
    
    if os.path.exists(output_path):
        print(f"Skipping {date_str} (already exists)")
    else:
        scrape_day(date_str)
    
    current += timedelta(days=1)


Saved 2022-07-21
Saved 2022-07-22
Saved 2022-07-23
Saved 2022-07-24
Saved 2022-07-25
Saved 2022-07-26
Saved 2022-07-27
Saved 2022-07-28
Saved 2022-07-29
Saved 2022-07-30
Saved 2022-07-31
Saved 2022-08-01
Saved 2022-08-02
Saved 2022-08-03
Saved 2022-08-04
Saved 2022-08-05
Saved 2022-08-06
Saved 2022-08-07
Saved 2022-08-08
Saved 2022-08-09
Saved 2022-08-10
Saved 2022-08-11
Saved 2022-08-12
Saved 2022-08-13
Saved 2022-08-14
Saved 2022-08-15
Saved 2022-08-16
Saved 2022-08-17
Saved 2022-08-18
Saved 2022-08-19
Saved 2022-08-20


In [ ]:
# so now, i want to get the win percentages cumulative over the season and then filter by the odds data that we have.